#### Question 1. Version of pandas.
2.0.3

In [2]:
!conda list pandas

# packages in environment at /home/sjanse/miniconda3/envs/mlzoomcamp:
#
# Name                    Version                   Build  Channel
pandas                    2.0.3           py310h1128e8f_0  


#### 2. Question 2: How many columns in the dataset?
10

In [4]:
!wget https://raw.githubusercontent.com/alexeygrigorev/datasets/master/housing.csv -O ./data/housing.csv

--2023-09-17 23:26:02--  https://raw.githubusercontent.com/alexeygrigorev/datasets/master/housing.csv
Loaded CA certificate '/etc/ssl/certs/ca-certificates.crt'
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1423529 (1,4M) [text/plain]
Saving to: ‘./data/housing.csv’

./data/housing.csv  100%[===================>]   1,36M  6,08MB/s    in 0,2s    

2023-09-17 23:26:02 (6,08 MB/s) - ‘./data/housing.csv’ saved [1423529/1423529]



In [5]:
import pandas as pd

df = pd.read_csv("./data/housing.csv")
print(df.shape)

(20640, 10)


#### Q3. Which columns of the dataset have missing values?
total_bedrooms

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20640 entries, 0 to 20639
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   longitude           20640 non-null  float64
 1   latitude            20640 non-null  float64
 2   housing_median_age  20640 non-null  float64
 3   total_rooms         20640 non-null  float64
 4   total_bedrooms      20433 non-null  float64
 5   population          20640 non-null  float64
 6   households          20640 non-null  float64
 7   median_income       20640 non-null  float64
 8   median_house_value  20640 non-null  float64
 9   ocean_proximity     20640 non-null  object 
dtypes: float64(9), object(1)
memory usage: 1.6+ MB


#### Q4. How many unique values does the ocean_proximity column have?
5

In [9]:
print(df.ocean_proximity.nunique())
print(df.ocean_proximity.unique())

5
['NEAR BAY' '<1H OCEAN' 'INLAND' 'NEAR OCEAN' 'ISLAND']


#### Q5. What's the average value of the median_house_value for the houses located near the bay?
259212

In [12]:
df[df.ocean_proximity == "NEAR BAY"].median_house_value.mean()

259212.31179039303

#### Q6. Has it changed?
No

In [13]:
d = df.copy()
initial = d.total_bedrooms.mean()
d.total_bedrooms = d.total_bedrooms.fillna(value=initial)
after = d.total_bedrooms.mean()
print(F"initial: {initial}; after: {after}; delta: {after - initial}")

initial: 537.8705525375618; after: 537.8705525375617; delta: -1.1368683772161603e-13


#### Q7. What's the value of the last element of w?
5.6992

In [26]:
import numpy as np


def vector_vector_multiplication(u, v):
    assert u.shape[0] == v.shape[0]
    
    n = u.shape[0]
    
    result = 0.0

    for i in range(n):
        result = result + u[i] * v[i]
    
    return result


def matrix_vector_multiplication(U, v):
    assert U.shape[1] == v.shape[0]
    
    num_rows = U.shape[0]
    
    result = np.zeros(num_rows)
    
    for i in range(num_rows):
        result[i] = vector_vector_multiplication(U[i], v)
    
    return result


def matrix_matrix_multiplication(U, V):
    assert U.shape[1] == V.shape[0]
    
    num_rows = U.shape[0]
    num_cols = V.shape[1]
    
    result = np.zeros((num_rows, num_cols))
    
    for i in range(num_cols):
        vi = V[:, i]
        Uvi = matrix_vector_multiplication(U, vi)
        result[:, i] = Uvi
    
    return result

In [61]:
islands = df[df.ocean_proximity == "ISLAND"].copy()[["housing_median_age", "total_rooms", "total_bedrooms"]]
X = islands.values

In [62]:
xtx = matrix_matrix_multiplication(X.T, X)
inverse_xtx = np.linalg.inv(xtx)

In [63]:
y = np.array([950, 1300, 800, 1000, 1300])

In [64]:
multiplication = matrix_matrix_multiplication(inverse_xtx, X.T)

In [67]:
multiplication * y

array([[-7.84328036, -8.49171012, -1.85727372, 25.65143741, 15.6641364 ],
       [-1.57559761,  1.44833145,  1.11724729, -1.03214974, -1.43907321],
       [ 7.16646682, -3.92552691, -3.64100058,  1.81685089,  4.28243922]])

In [70]:
w = matrix_vector_multiplication(multiplication, y)

In [71]:
w

array([23.12330961, -1.48124183,  5.69922946])